In [55]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

speech_links = pd.read_csv(r'C:\Users\FYD\Documents\GitHub\lda-brazilian-books\src\books\discursos-presidenciais\discursos-presidenciais-links.txt',
                             delimiter='|')

speech_links.head(2)

,president,link
0,fhc,/pt-BR/discursos-artigos-e-entrevistas-categor...
1,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...


In [57]:
start = time.time()

BASE_URL = 'http://www.itamaraty.gov.br'

full_link = []
speech = []
for link in speech_links['link']:
    full_link.append(BASE_URL + link)
    
    response = requests.get(BASE_URL + link)
    soup = BeautifulSoup(response.text, 'html.parser')
    speech_text = soup.find("div", itemprop="articleBody")
    
    speech.append(speech_text)
    
print ("Process time: " + str((time.time() - start)))

Process time: 407.8306894302368


In [164]:
df_speech_consolidated = pd.concat([speech_links,
                                   pd.DataFrame(full_link),
                                   pd.DataFrame(speech)], axis=1)

df_speech_consolidated.columns = ['president', 'link', 'full_link', 'text']
df_speech_consolidated.head(5)

,president,link,full_link,text
0,fhc,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <div class=""plain..."
1,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
2,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
3,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
4,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."


In [165]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]"\|@,;]')

from nltk.corpus import stopwords

#BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('portuguese'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = text.replace("<div itemprop=\"articleBody\">", "")
    text = text.replace("<div class=\"plain\" id=\"parent-fieldname-text\">", "")
    text = text.replace("<p style=\"text-align: justify;\">", "")
    text = text.replace("<br/>", "")
    text = text.replace("</div>", "")
    text = text.replace("\n", "")
    text = text.replace(" < p>", "")    
    text = text.replace(".< p>", " ")
    text = text.replace(".", " ")
    text = text.replace("<div itemprop=\"articlebody\">", "")
    text = text.replace("<", "")
    text = text.replace(">", "")
     
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    #text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
    
df_speech_consolidated['text_processed'] = df_speech_consolidated['text'].apply(clean_text)

In [166]:
df_speech_consolidated.to_csv(r'C:\Users\FYD\Documents\GitHub\lda-brazilian-books\src\books\discursos-presidenciais\df_speech_consolidated.csv',
                             sep='|',
                             index_label=None)

In [167]:
df_speech_consolidated['text_processed'].head(1)[0]

'desejo antes tudo agradecer presidente jorge sampaio esposa maria josé ritta gesto receber palácio ajuda demonstração fidalguia distingue estimado casal amigo brasil amigo brasileiros convivência presidente sampaio faz nutrir apreço cada vez maior sítios históricos portugal sinto cativo palácio ajuda onde evoca nascimento rei comum d joão vi percebo próximo palácio bélem hospedou passagem ano mostra sobre carta pêro vaz caminha cujo catálogo satisfação prefaciar juntamente presidente sampaio ressaltei prefácio prestígio hoje cerca carta dom manuel i decorre apenas dotes narrativos escrivão resulta sobretudo fato encontro etnias descrito caminha vingou vingou bem dando margem universo civilizatório profícuos caracterizado ambos lados atlântico plasticidade cultural propensão assimilar influências diversos povos sobre atualidade lastro cultural une brasil portugal gostaria dizer algumas palavras hoje noite inspirado anfitriões jorge sampaio maria ritta tanto cultivam legado humanista ja